In [1]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
pip install seaborn

   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ------------ --------------------------- 92.2/294.9 kB 2.6 MB/s eta 0:00:01
   ------------------------------- -------- 235.5/294.9 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Input, Concatenate, ZeroPadding2D, LSTM
from keras.layers import Conv2D
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split  
import numpy as np
import os
import random
import cv2
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Densenet201 Model Definition
def densenet201_model(img_rows, img_cols, color_type=1, nb_dense_block=4, growth_rate=32, nb_filter=64, 
                      reduction=0.5, dropout_rate=0.0, weight_decay=1e-4, num_classes=None):
    '''
    DenseNet 201 Model for Keras
    '''
    global concat_axis
    img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
    concat_axis = 3

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6, 12, 48, 32]  # For DenseNet-201

    # Initial convolution
    x = Conv2D(nb_filter, (7, 7), strides=(2, 2), padding='same', name='conv1', use_bias=False)(img_input)
    x = BatchNormalization(axis=concat_axis)(x)
    # Commenting Scale, ensure custom scale layer is available
    # x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx + 2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate)
        x = transition_block(x, stage, nb_filter, dropout_rate=dropout_rate)
        nb_filter = int(nb_filter)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate)
    x = BatchNormalization(axis=concat_axis)(x)
    # x = Scale(axis=concat_axis)(x) # Optional: Use if Scale layer is defined
    x = Activation('relu')(x)

    x_fc = GlobalAveragePooling2D()(x)
    x_fc = Dense(num_classes)(x_fc)
    x_fc = Activation('softmax')(x_fc)

    model = Model(img_input, x_fc)

    # Compile the model
    sgd = SGD(learning_rate=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


# Helper functions for Dense Blocks, Transition Blocks, etc.
def conv_block(x, stage, branch, nb_filter, dropout_rate=None):
    inter_channel = nb_filter * 4  
    x = BatchNormalization(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Conv2D(inter_channel, (1, 1), use_bias=False)(x)  # Updated here

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = BatchNormalization(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D((1, 1))(x)
    x = Conv2D(nb_filter, (3, 3), use_bias=False)(x)  # Updated here

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x

def transition_block(x, stage, nb_filter, dropout_rate=None):
    x = BatchNormalization(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(nb_filter), (1, 1), use_bias=False)(x)  # Updated here

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, grow_nb_filters=True):
    '''Build a dense block where the output of each conv_block is fed to subsequent ones'''
    concat_feat = x
    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate)
        concat_feat = Concatenate(axis=concat_axis)([concat_feat, x])

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter


# Data Loading for Medicinal Leaf Dataset
def load_data():
    data = []
    labels = []
    random.seed(42)
    
    dataset_path = "Medicinal Leaf Dataset/Segmented Medicinal Leaf Images/"
    class_folders = sorted(list(os.listdir(dataset_path)))
    
    for class_name in class_folders:
        class_path = os.path.join(dataset_path, class_name)
        for img_name in sorted(os.listdir(class_path)):
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (224, 224))
            image = img_to_array(image)
            data.append(image)
            labels.append(class_name)
    
    data = np.array(data, dtype="float32") / 255.0
    labels = np.array(labels)
    
    # Convert labels to binary format
    mlb = LabelBinarizer()
    labels = mlb.fit_transform(labels)
    
    # Split the data into training and testing sets
    (xtrain, xtest, ytrain, ytest) = train_test_split(data, labels, test_size=0.25, random_state=42)
    
    return xtrain, ytrain, xtest, ytest


# Main Script
if __name__ == '__main__':
    img_rows, img_cols = 224, 224  # Image dimensions
    channel = 3                    # Number of color channels
    num_classes = 30               # Number of classes (30 leaf categories)
    batch_size = 16
    nb_epoch = 10

    # Load the dataset
    X_train, Y_train, X_valid, Y_valid = load_data()

    # Initialize and compile DenseNet201 model
    model = densenet201_model(img_rows=img_rows, img_cols=img_cols, color_type=channel, num_classes=num_classes)
    
    # Model checkpoint to save the best model
    filepath = "bestmodel_densenet201.keras"  # Change .hdf5 to .keras
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    # Train the model
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, shuffle=True, verbose=1,
                        validation_data=(X_valid, Y_valid), callbacks=callbacks_list)

    # Plotting training results
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.legend()
    plt.show()


    # Evaluate accuracy
    xtest = X_valid
    ytest = Y_valid
    ypred = model.predict(xtest)
    total = 0
    accurate = 0
    for i in range(len(ypred)):
        if np.argmax(ypred[i]) == np.argmax(ytest[i]):
            accurate += 1
        total += 1

    print('Accuracy: ', (accurate / total) * 100)


Epoch 1/10
 1/86 ━━━━━━━━━━━━━━━━━━━━ 2:48:02 119s/step - accuracy: 0.0625 - loss: 3.5200